Download Schema

In [ ]:
import requests
import os
import pandas as pd

# Set up the logger
import logging

logger = logging.getLogger()

def download(url, filename=None):
  """Downloads a file from a URL and saves it locally.

  Args:
    url: The URL of the file to download.
    filename: The filename to save the downloaded file as. If not specified,
      the filename will be extracted from the URL.
  """
  response = requests.get(url, stream=True)
  response.raise_for_status()
  
  if filename is None:
    filename = url.split("/")[-1]

  if os.path.isfile(filename):
    os.remove(filename)  

  if os.path.isfile(filename+".wal"):
    os.remove(filename+".wal")  

  with open(filename, "wb") as f:
    for chunk in response.iter_content(1024):
      if chunk:  # filter out keep-alive new chunks
        f.write(chunk)
        
  for x in os.listdir():
    if x.endswith(".db") or x.endswith(".wal"):
        # Prints only text file present in My Folder
        print(x)
        
  return filename

In [ ]:
import os
database_name = "finance"
filename = f"/tmp/{database_name}.db"
if not os.path.isfile(filename):
    url = f"http://test-public-storage-440955376164.s3-website.us-east-1.amazonaws.com/catalogs/{database_name}.db"
    filename = download(url, filename)    

Open Database and Login

In [ ]:
from . import login
connection = login.cognito("vaultdb", "test123", filename, aws_region="us-east-1")

In [ ]:
connection.execute("SELECT * FROM vaultdb_configs").fetchdf()

In [ ]:
connection.execute("select * from duckdb_secrets();").fetch_df()

In [ ]:
connection.execute("select * from information_schema.tables").fetchdf()

In [ ]:
connection.execute("SET timezone='America/New_York';")
connection.execute("select * from quote where date='2024-06-24'").fetchdf()

In [ ]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM tickers").fetchdf()

Data Inheritance

In [ ]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM demo").fetchdf()

Insert Data Locally

In [ ]:
connection.execute("INSERT INTO demo VALUES (81,92,93,94,95,96,97,98)")

View Local Data

In [ ]:
connection.execute(f"PRAGMA disable_data_inheritance;")
connection.execute("SELECT * FROM demo").fetchdf()

Enable Inheritance

In [ ]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM demo").fetchdf()

Push local Data

In [ ]:
connection.execute(f'PUSH DATABASE {database_name};').fetchdf()

Truncate Local data

In [ ]:
connection.execute(f'TRUNCATE DATABASE {database_name};').fetchdf()

View remote Data

In [ ]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM demo").fetchdf()

Create Objects Locally

In [ ]:
connection.execute('BEGIN TRANSACTION;')
connection.execute('CREATE TABLE demo2 (col1 INT, col2 INT, col3 INT, col4 INT, col5 INT, col6 INT, col7 INT, col8 INT)')
connection.execute("INSERT INTO demo2 VALUES (1,2,3,4,5,6,7,8), (11,22,33,44,55,66,77,88), (111,222,333,444,555,666,777,888), (1111,2222,3333,4444,5555,6666,7777,8888)")
connection.execute('COMMIT;')

Push Schema Changes on to server

In [ ]:
connection.execute('PUSH DATABASE {database_name};').fetchdf()